# Using R in Teaching from *Network Science*

### Amir Barghi, Department of Mathematics and Statistics, Saint Michael's College

----

# Chap. 4: The Scale Free Property

In [ ]:
# configuring figure size
options(repr.plot.width = 6, repr.plot.height = 4)

# Loading Packages

In [ ]:
# uncomment the following lines if you have not installed qdread/forestscaling; for more information, see https://github.com/qdread/forestscaling/

#install.packages('remotes')
#remotes::install_github('qdread/forestscaling')

In [ ]:
library(tidyverse)

library(igraph)

library(igraphdata)

library(ggraph)

library(e1071)

library(forestscaling)

library(latex2exp)

# Loading the Data Set 

## Data from [`igraphdata::yeast`](https://github.com/igraph/igraphdata)

Data Source: von Mering, C., Krause, R., Snel, B. et al. Comparative assessment of large-scale data sets of protein–protein interactions. *Nature* **417**, 399–403 (2002). https://doi.org/10.1038/nature750

## Yeast Protein Interaction Network

In [ ]:
data(yeast)

g <- yeast

In [ ]:
set.seed(42)

ggraph(g, layout = 'mds') + 
  geom_edge_fan(edge_linetype = 3, color = 'dark blue', alpha = 0.25) + 
  geom_node_point(color = 'dark red', size = .75, alpha = 0.75) + 
  theme_graph(base_family = 'Helvetica') +
  labs(title = 'Yeast Interaction Network',
       subtitle = 'Displayed Using Multidimensional Scaling Layout')

# 2.11 Summary

In [ ]:
suppressMessages(df <- bind_cols(enframe(eccentricity(g)), 
                                 enframe(betweenness(g)), 
                                 enframe(degree(g)), 
                                 enframe(transitivity(g, type = c('local')))))

df <- df %>% select(name...1, value...2, value...4, value...6, value...8)

names(df) <- c('name', 'eccentricity', 'betweenness', 'degree', 'clustering')

head(df)

tail(df)

glimpse(df)

In [ ]:
df %>% 
  summarize(avg_deg = mean(degree), 
            delta = max(degree), 
            prop = sum(degree <= avg_deg) / n(), 
            diam = max(eccentricity),
            radius = min(eccentricity),
            avg_cc = mean(clustering, na.rm = TRUE),
            avg_distance = mean_distance(g, directed = FALSE, unconnected = TRUE))

# 4.3 Hubs

In [ ]:
df %>% 
  ggplot(aes(x = degree, y = ..density..)) + 
  geom_density(fill = 'red') + 
  labs(title = 'KDE of Degrees in the Yeast Data Set')

In [ ]:
df %>% 
  ggplot(aes(x = degree, y = ..density..)) + 
  geom_histogram(binwidth = 1, fill = 'blue') +
  labs(title = 'Histogram of Degrees in the Yeast Data Set')

In [ ]:
# the top 20 degree values
df %>% arrange(desc(degree)) %>% head(20)

In [ ]:
df %>%
  summarize(avg_deg = mean(df$degree),
            sd_deg = sd(df$degree),
            second_moment_deg = moment(df$degree, 2),
            var_deg = moment(df$degree, 2) - moment(df$degree,1)**2,
            sd_deg_formula = sqrt(moment(df$degree, 2) - moment(df$degree,1)**2))

# 4.5 Universality and 4.13 Advanced Topics 4.C: Estimating the Degree Exponent

In [ ]:
(pl.fit <- fit_power_law(df$degree, implementation = 'plfit'))

In [ ]:
fit_power_law(df$degree, implementation = 'plfit', force.continuous = TRUE)

In [ ]:
# maximum likelihood optimizing function
fit_power_law(df$degree, implementation = 'R.mle')

In [ ]:
# maximum likelihood optimizing function
fit_power_law(df$degree, xmin = 13, implementation = 'R.mle')

In [ ]:
total <- sum(df$degree)

df %>% 
  group_by(degree) %>% 
  mutate(k = degree,  p_k = n() / gorder(g)) %>% 
  ungroup() %>%
  ggplot(aes(x = k, y = p_k)) + 
  geom_point(color = 'blue', size = .5) + 
  scale_x_log10() + 
  scale_y_log10() +
  labs(title = 'Log-Log Scale, Linear Binning for Degree Distribution',
       subtitle = 'in the Yeast Data Set') +
  labs(y = TeX('$p_k$'))

# 4.12 Advanced Topics 4.B: Plotting Power Laws Degree Sequence Scatter Plot

In [ ]:
df %>% 
  group_by(degree) %>% 
  mutate(k = degree,  count = n()) %>% 
  ungroup() %>%
  ggplot(aes(x = k, y = count)) + 
  geom_point(color = 'red', size = .5) +
  labs(title = 'Degree Distribution (Counts) in the Yeast Data Set')

In [ ]:
df %>% 
  group_by(degree) %>% 
  mutate(k = degree,  p_k = n() / gorder(g)) %>% 
  ungroup() %>%
  ggplot(aes(x = k, y = p_k)) + 
  geom_point(color = 'blue', size = .5) + 
  labs(title = 'Degree Distribution (Proportions) in the Yeast Data Set') +
  labs(y = TeX('$p_k$'))

### Fig. 4.22(b) Log-Log Scale, Linear Binning

In [ ]:
df %>% 
  group_by(degree) %>% 
  mutate(k = degree,  p_k = n() / gorder(g)) %>% 
  ungroup() %>%
  ggplot(aes(x = k, y = p_k)) + 
  geom_point(color = 'blue', size = .5) + 
  geom_line(aes(x = degree, y = degree ** (-pl.fit$alpha)), 
            color = 'red') + 
  scale_x_log10() + 
  scale_y_log10() +
  labs(y = TeX('$p_k$')) +
  labs(title = 'Log-Log Scale, Linear Binning for Degree Distribution',
       subtitle = 'with Fitted Power Law Curve in the Yeast Data Set')

### Fig. 4.22(c) Log-Log Scale, Logarithmic Binning

In [ ]:
df2 <- logbin(df$degree, n = 100) #change n to 10, 15, 20, 25, 50, 75 to compare

df2

total2 <- sum(df2$bin_value)

In [ ]:
df2 %>% 
  filter(bin_count != 0) %>% 
  ggplot(aes(x = bin_midpoint, y = bin_value / total2)) + 
  geom_point(color = 'blue', size = .5) + 
  geom_line(data = df, 
            aes(x = degree, y = degree ** (-pl.fit$alpha)), 
            color = 'red') + 
  scale_x_log10() + 
  scale_y_log10() +
  labs(y = TeX('$p_k$'), x = 'k') +
  labs(title = 'Log-Log Scale, Logarithmic Binning for Degree Distribution',
       subtitle = 'with Fitted Power Law Curve in the Yeast Data Set')

# 4.6 Ultra-Small-World Property

In [ ]:
mean_distance(g, directed = FALSE, unconnected = TRUE)

log(log(vcount(g)))

# 4.8 Generating Networks with Arbitrary Degree Distribution

## 4.8.1 Configuration Model

In [ ]:
set.seed(42)

h <- sample_degseq(df$degree, method = 'simple.no.multiple') # method = c('simple', 'vl', 'simple.no.multiple')

In [ ]:
set.seed(42)

ggraph(h, layout = 'mds') + 
  geom_edge_fan(edge_linetype = 3, color = 'dark blue', alpha = 0.25) + 
  geom_node_point(color = 'dark red', size = .75, alpha = 0.75) + 
  theme_graph(base_family = 'Helvetica') +
  labs(title = 'A Reconfigured Yeast Interaction Network',
       subtitle = 'Displayed Using Multidimensional Scaling Layout')

In [ ]:
suppressMessages(df3 <- bind_cols(enframe(eccentricity(h)), 
                                  enframe(betweenness(h)), 
                                  enframe(degree(h)), 
                                  enframe(transitivity(h, type = c('local')))))

df3 <- df3 %>% select(name...1, value...2, value...4, value...6, value...8)

names(df3) <- c('name', 'eccentricity', 'betweenness', 'degree', 'clustering')

In [ ]:
df3 %>% 
  summarize(avg_deg = mean(degree), 
            delta = max(degree), 
            prop = sum(degree <= avg_deg) / n(), 
            diam = max(eccentricity),
            radius = min(eccentricity),
            avg_cc = mean(clustering, na.rm = TRUE))

In [ ]:
df3 %>% 
  ggplot(aes(x = degree, y = ..density..)) + 
  geom_density(fill = 'red') +
  labs(title = 'KDE of Degrees in the Reconfigured Yeast Data Set')

In [ ]:
df3 %>% 
  ggplot(aes(x = degree, y = ..density..)) + 
  geom_histogram(binwidth = 1, fill = 'blue') +
  labs(title = 'Histogram of Degrees in the Reconfigured Yeast Data Set')

In [ ]:
(pl.fit2 <- fit_power_law(df3$degree, implementation = 'plfit'))

In [ ]:
df3 %>% 
  group_by(degree) %>% 
  mutate(k = degree,  p_k = n() / gorder(h)) %>% 
  ungroup() %>%
  ggplot(aes(x = k, y = p_k)) + 
  geom_point(color = 'blue', size = .5) +
  labs(title = 'Degree Distribution (Proportions) in the Reconfigured Yeast Data Set') +
  labs(y = TeX('$p_k$'))

In [ ]:
df4 <- logbin(df3$degree, n = 100)

total4 <- sum(df4$bin_value)

df4 %>% 
  filter(bin_count != 0) %>% 
  ggplot(aes(x = bin_midpoint, y = bin_value / total4)) + 
  geom_point(color = 'blue', size = .5) + 
  geom_line(data = df3, 
            aes(x = degree, y = degree ** (-pl.fit2$alpha)), 
            color = 'red') + 
  scale_x_log10() + 
  scale_y_log10() +
  labs(y = TeX('$p_k$'), x = 'k') +
  labs(title = 'Log-Log Scale, Logarithmic Binning for Degree Distribution',
       subtitle = 'with Fitted Power Law Curve in the Reconfigured Yeast Data Set')


## 4.8.2 Degree-preserving Randomization

In [ ]:
set.seed(42)

data(yeast)

f <- yeast

f <- rewire(f, keeping_degseq(niter = gorder(f) * 10))

In [ ]:
set.seed(42)

ggraph(f, layout = 'mds') + 
  geom_edge_fan(edge_linetype = 3, color = 'dark blue', alpha = 0.25) + 
  geom_node_point(color = 'dark red', size = .75, alpha = 0.75) + 
  theme_graph(base_family = 'Helvetica') +
  labs(title = 'A Randomized Yeast Interaction Network',
       subtitle = 'Displayed Using Multidimensional Scaling Layout')

In [ ]:
suppressMessages(df5 <- bind_cols(enframe(eccentricity(f)), 
                                  enframe(betweenness(f)), 
                                  enframe(degree(f)), 
                                  enframe(transitivity(f, type = c('local')))))

df5 <- df5 %>% select(name...1, value...2, value...4, value...6, value...8)

names(df5) <- c('name', 'eccentricity', 'betweenness', 'degree', 'clustering')

In [ ]:
df5 %>% 
  summarize(avg_deg = mean(degree), 
            delta = max(degree), 
            prop = sum(degree <= avg_deg) / n(), 
            diam = max(eccentricity), 
            radius = min(eccentricity),
            avg_cc = mean(clustering, na.rm = TRUE))

In [ ]:
df5 %>% 
  ggplot(aes(x = degree, y = ..density..)) + 
  geom_density(fill = 'red') +
  labs(title = 'KDE of Degrees in the Randomized Yeast Data Set')

In [ ]:
df5 %>% 
  ggplot(aes(x = degree, y = ..density..)) + 
  geom_histogram(binwidth = 1, fill = 'blue') + 
  labs(title = 'Histrogram of Degrees in the Randomized Yeast Data Set')

In [ ]:
(pl.fit3 <- fit_power_law(df5$degree, implementation = 'plfit'))

In [ ]:
df5 %>% 
  group_by(degree) %>% 
  mutate(k = degree,  p_k = n() / gorder(f)) %>%
  ungroup() %>%
  ggplot(aes(x = k, y = p_k)) + 
  geom_point(color = 'blue', size = .5) + 
  labs(title = 'Degree Distribution (Proportions) in the Randomized Yeast Data Set') +
  labs(y = TeX('$p_k$'))

In [ ]:
df6 <- logbin(df5$degree, n = 100)

total6 <- sum(df6$bin_value)

df6 %>% filter(bin_count != 0) %>% 
  ggplot(aes(x = bin_midpoint, y = bin_value/total6)) + 
  geom_point(color = 'blue', size = .5) + 
  geom_line(data = df5, 
            aes(x = degree, y = degree ** (-pl.fit3$alpha)), 
            color = 'red') + 
  scale_x_log10() + 
  scale_y_log10() +
  labs(y = TeX('$p_k$'), x = 'k') +
  labs(title = 'Log-Log Scale, Logarithmic Binning for Degree Distribution',
       subtitle = 'with Fitted Power Law Curve in the Randomized Yeast Data Set')

# References

- Albert-Laszlo Barabasi, *Network Science*, Cambridge University Press, 2016. *Network Science* is available online at http://networksciencebook.com/ under the following license: "This book's text and illustrations are licensed under a [Creative Commons Attribution-NonCommercial 3.0 Unported License](https://creativecommons.org/licenses/by-nc/3.0/)."
- von Mering, C., Krause, R., Snel, B. et al. Comparative assessment of large-scale data sets of protein–protein interactions. *Nature* **417**, 399–403 (2002). https://doi.org/10.1038/nature750